In [1]:
import sim
import numpy as np
import math
import time

def connect(port):
# Establece la conexión a COPPELIA
# El port debe coincidir con el puerto de conexión en VREP  -- DALE AL PLAY !!!
# retorna el número de cliente o -1 si no puede establecer conexión
    sim.simxFinish(-1) # just in case, close all opened connections
    clientID=sim.simxStart('127.0.0.1',port,True,True,2000,5) # Conectarse
    if clientID == 0: print("conectado a", port)
    else: print("no se pudo conectar")
    return clientID

# Conectarse al servidor de COPPELIA
# *** _Hay que ejecutarlo cada vez que se reinicia la simulación ***
clientID = connect(19999)
ret,nema23=sim.simxGetObjectHandle(clientID,'nema23',sim.simx_opmode_blocking)

conectado a 19999


In [2]:
MAX_DEG = 60
MIN_DEG = 0
MAX_VEL = 1
STOP = 0

VOL_MIN=50
VOL_MAX=90

RPM_MIN=5
RPM_MAX=30

RADI_MIN=1
RADI_MAX=3

ERROR_MESSAGE= "Valor incorrecte. Siusplau, introdueixi un valor entre"

# Per assigar els parametres del robot, sha utilitzat criteris aproximats, logics i proporcionals als angles maxims i velocitat 
# maxims del robot. PEr fer'ho realista i segur, hauriem de coneixer les caracteristiques mediques dún ventilador manual extern, 
# quines son les necessitats pulmonars del cos humà

def movimentParametritzat(volAire, rpm, radi):
    angle=0 

    maxDeg = volAire-30
 
    vel = ((rpm-RPM_MIN)*(100/(RPM_MAX-RPM_MIN)))/100

    velInsp = vel
    velExp = vel
    
    if radi != RADI_MIN:
        if radi == RADI_MAX:
            velInsp = vel*0.5
        else:
            velExp = vel*0.5  
            
    ## - Funcionalitat - ##
    #for i in range(20):
        while(angle<maxDeg):
            sim.simxSetJointTargetVelocity(clientID, nema23, velInsp, sim.simx_opmode_oneshot)
            retCode,radNema = sim.simxGetJointPosition(clientID, nema23, sim.simx_opmode_blocking)
            angle=radNema *180/np.pi

        sim.simxSetJointTargetVelocity(clientID, nema23, STOP, sim.simx_opmode_oneshot)

        while(angle>MIN_DEG):
            sim.simxSetJointTargetVelocity(clientID, nema23, -velExp, sim.simx_opmode_oneshot)
            retCode,radNema = sim.simxGetJointPosition(clientID, nema23, sim.simx_opmode_blocking)
            angle=radNema *180/np.pi

        sim.simxSetJointTargetVelocity(clientID, nema23, STOP, sim.simx_opmode_oneshot)

In [3]:
import random
def getSPO2(aux):
    if aux == 1:#Comportamiento normal
        PSO2 = random.randint(95,99)
    elif aux == 2:
        PSO2 = random.randint(90,99)#Falta de oxigeno
    elif aux == 3:
        PSO2 = random.randint(95,100)#Exceso de oxigeno
    else: 
        print("Valor incorrecto")
        PSO2 = 0
    return PSO2

In [4]:
def setValues():
    vAire = random.randint(50,90)
    RPM = random.randint(15,30)
    radi = random.randint(1,3)
    return vAire,RPM,radi

def funcionOximetre(resNext, perOximetre):
    if (perOximetre < 95):
        if resNext < 40:
            print("Aumentan RPM")
            resNext = resNext + 1
    else:
        if resNext > 10:
            print("Disminuint RPM")
            resNext = resNext - 1
    return resNext

In [6]:
    ##########FUNCIO INTRODUIR PARAMETRES (POTENCIOMETRE)################
    vAire, res, radi = setValues()
    #####################################################################
    ##########FUNCIO Get SOP2 (Oximetre)################
    perOximetre = getSPO2(1)# 1- Valors Normals, 2- Falta d'oxigen, 3- Exces d'oxigen
    #####################################################################
    vAireNext = vAire
    resNext = res
    radiNext = radi

    counter = 0
    
    run = True
    surt = True

    while run:
        movimentParametritzat(vAire, res, radi)
        while(vAire == vAireNext and res == resNext and radi == radiNext and surt):
            print(vAire, res, radi)
            print(vAireNext, resNext, radiNext)
            print("Valors normals")
            if(perOximetre < 95 or perOximetre > 99):
                print("Problema d'Oxigen")
                resNext = funcionOximetre(resNext, perOximetre)
                surt = False
                print(resNext)
            movimentParametritzat(vAire, resNext, radi)
            perOximetre = getSPO2(2)
        #Si el usuario introdugera nuebos parametros se deverian actualizar en este punto
        vAire = vAireNext
        res = resNext
        radi = radiNext
        surt = True

        if counter == 20:
            run = False
            print(counter)
        else:
            counter += 1
            
    print("Final programa")
        

65 30 3
65 30 3
Valors normals
65 30 3
65 30 3
Valors normals
65 30 3
65 30 3
Valors normals
65 30 3
65 30 3
Valors normals
65 30 3
65 30 3
Valors normals
65 30 3
65 30 3
Valors normals
65 30 3
65 30 3
Valors normals
65 30 3
65 30 3
Valors normals
Problema d'Oxigen
Aumentan RPM
31
65 31 3
65 31 3
Valors normals
Problema d'Oxigen
Aumentan RPM
32
65 32 3
65 32 3
Valors normals
65 32 3
65 32 3
Valors normals
65 32 3
65 32 3
Valors normals
Problema d'Oxigen
Aumentan RPM
33
65 33 3
65 33 3
Valors normals
Problema d'Oxigen
Aumentan RPM
34
65 34 3
65 34 3
Valors normals
65 34 3
65 34 3
Valors normals
Problema d'Oxigen
Aumentan RPM
35
65 35 3
65 35 3
Valors normals
Problema d'Oxigen
Aumentan RPM
36
65 36 3
65 36 3
Valors normals
65 36 3
65 36 3
Valors normals
Problema d'Oxigen
Aumentan RPM
37
65 37 3
65 37 3
Valors normals
Problema d'Oxigen
Aumentan RPM
38
65 38 3
65 38 3
Valors normals
Problema d'Oxigen
Aumentan RPM
39


KeyboardInterrupt: 